# Determine Optimal Fantasy Auction Draft Price per Player
Luke Brown | Dylan DiGoia | Shawn Killpack | Sean Olson

In [2]:
# import libraries
# import standard libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import statsmodels.api as sm
from scipy import stats
from scipy.stats import skew
from scipy import interp

%matplotlib inline

# import statistical visualization library
import seaborn as sns

# import supervised learning libraries
from sklearn.model_selection import train_test_split, cross_val_score, RandomizedSearchCV
from imblearn.over_sampling import SMOTE
from sklearn.feature_selection import RFE, RFECV
from sklearn import model_selection
from sklearn.ensemble import RandomForestClassifier

# import regression libraries
from sklearn.linear_model import LinearRegression, Lasso, LogisticRegression
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, confusion_matrix, classification_report, roc_auc_score, roc_curve

# suppress warnings
pd.options.mode.chained_assignment = None
from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)

In [3]:
# import data files
av = pd.read_csv('./data/ProFootballReference_Fantasy_Approximate_Value.csv')
print('Approximate Value')
print('------------------')
display(av.head())

dem = pd.read_csv('./data/ProFootballReference_Fantasy_Demographics.csv')
print('Demographics')
print('-------------')
display(dem.head())

pas = pd.read_csv('./data/ProFootballReference_Fantasy_Passing.csv')
print('Passing')
print('---------')
display(pas.head())

rec = pd.read_csv('./data/ProFootballReference_Fantasy_Receiving.csv')
print('Receiving')
print('----------')
display(rec.head())

reca = pd.read_csv('./data/ProFootballReference_Fantasy_Receiving_Advanced.csv')
print('Receiving Advanced')
print('-------------------')
display(reca.head())

rush = pd.read_csv('./data/ProFootballReference_Fantasy_Rushing.csv')
print('Rushing')
print('---------')
display(rush.head())

rusha = pd.read_csv('./data/ProFootballReference_Fantasy_Rushing_Advanced.csv')
print('Rushing Advanced')
print('-----------------')
display(rusha.head())

scor = pd.read_csv('./data/ProFootballReference_Fantasy_Scoring.csv')
print('Scoring')
print('---------')
display(scor.head())


Approximate Value
------------------


,_type,age,all_pros_first_team,av,g,gs,league_id,player,pro_bowls,psl_draft_info,seasons,team,year_id
0,dict,27.0,1,26.0,16.0,16.0,NFL,LaDainian Tomlinson,1,1-5,1,SDG,2006
1,dict,30.0,1,24.0,16.0,16.0,NFL,Tom Brady,1,6-199,1,NWE,2007
2,dict,29.0,1,23.0,14.0,14.0,NFL,Priest Holmes,1,NaN,1,KAN,2002
3,dict,28.0,1,23.0,15.0,15.0,NFL,Aaron Rodgers,1,1-24,1,GNB,2011
4,dict,27.0,1,22.0,14.0,14.0,NFL,Marshall Faulk,1,1-2,1,STL,2000


Demographics
-------------


,_type,age,bmi,g,gs,height_in,league_id,player,psl_draft_info,team,weight,year_id
0,dict,24.0,33.8,9.0,0.0,6-9,NFL,Demar Dotson,NaN,TAM,315.0,2009
1,dict,26.0,33.8,13.0,2.0,6-9,NFL,Demar Dotson,NaN,TAM,315.0,2011
2,dict,27.0,33.8,16.0,15.0,6-9,NFL,Demar Dotson,NaN,TAM,315.0,2012
3,dict,28.0,33.8,16.0,16.0,6-9,NFL,Demar Dotson,NaN,TAM,315.0,2013
4,dict,29.0,33.8,16.0,16.0,6-9,NFL,Demar Dotson,NaN,TAM,315.0,2014


Passing
---------


...

,_type,age,fantasy_points,fantasy_points_per_game,fantasy_points_ppr,fantasy_points_ppr_per_game,fumbles,g,gs,league_id,...,player,psl_draft_info,rec,rec_td,rec_yds,rush_att,rush_td,rush_yds,team,year_id
0,dict,27.0,427.1,26.69,483.1,30.19,2.0,16.0,16.0,NFL,...,LaDainian Tomlinson,1-5,56.0,3.0,508.0,348.0,28.0,1815.0,SDG,2006
1,dict,23.0,417.1,26.07,417.1,26.07,9.0,16.0,16.0,NFL,...,Patrick Mahomes,1-10,0.0,0.0,0.0,60.0,2.0,272.0,KAN,2018
2,dict,37.0,412.0,25.75,412.0,25.75,11.0,16.0,16.0,NFL,...,Peyton Manning,1-1,0.0,0.0,0.0,32.0,1.0,-31.0,DEN,2013
3,dict,30.0,398.0,24.88,398.0,24.88,6.0,16.0,16.0,NFL,...,Tom Brady,6-199,0.0,0.0,0.0,37.0,2.0,98.0,NWE,2007
4,dict,28.0,397.4,26.49,397.4,26.49,5.0,15.0,15.0,NFL,...,Aaron Rodgers,1-24,0.0,0.0,0.0,60.0,3.0,257.0,GNB,2011


Rushing Advanced
-----------------


,_type,age,g,gs,league_id,player,psl_draft_info,rush_att,rush_td,rush_yds,rush_yds_per_att,rush_yds_per_g,team,year_id
0,dict,27.0,16.0,16.0,NFL,Adrian Peterson,1-7,348.0,12.0,2097.0,6.03,131.1,MIN,2012
1,dict,24.0,16.0,16.0,NFL,Jamal Lewis,1-5,387.0,14.0,2066.0,5.34,129.1,BAL,2003
2,dict,24.0,16.0,16.0,NFL,Chris Johnson,1-24,358.0,14.0,2006.0,5.60,125.4,TEN,2009
3,dict,26.0,16.0,16.0,NFL,Ahman Green,3-76,355.0,15.0,1883.0,5.30,117.7,GNB,2003
4,dict,28.0,16.0,16.0,NFL,Shaun Alexander,1-19,370.0,27.0,1880.0,5.08,117.5,SEA,2005


Scoring
---------


,_type,age,all_td,fg_perc,fga,fgm,g,gs,league_id,player,psl_draft_info,safety_md,scoring,team,two_pt_md,xp_perc,xpa,xpm,year_id
0,dict,27.0,31.0,NaN,NaN,NaN,16.0,16.0,NFL,LaDainian Tomlinson,1-5,NaN,186.0,SDG,NaN,NaN,NaN,NaN,2006
1,dict,28.0,28.0,NaN,NaN,NaN,16.0,16.0,NFL,Shaun Alexander,1-19,NaN,168.0,SEA,NaN,NaN,NaN,NaN,2005
2,dict,30.0,27.0,NaN,NaN,NaN,16.0,16.0,NFL,Priest Holmes,NaN,NaN,162.0,KAN,NaN,NaN,NaN,NaN,2003
3,dict,27.0,26.0,NaN,NaN,NaN,14.0,14.0,NFL,Marshall Faulk,1-2,NaN,160.0,STL,2.0,NaN,NaN,NaN,2000
4,dict,29.0,24.0,NaN,NaN,NaN,14.0,14.0,NFL,Priest Holmes,NaN,NaN,144.0,KAN,NaN,NaN,NaN,NaN,2002


In [ ]:
# aggregate scoring, demographic and approximate value datasets by player into rushing, passing, and receiving
